In [1]:
import matplotlib.pyplot as plt
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
from lwa_f import blocks
#from lwautils import lwa_arx 
import pandas as pd
from cr_functions import *
import struct
import math
from scipy import special as sp
from scipy import optimize as oz
from scipy import stats as st
from lwa_antpos import mapping
import os



dsautils.cnf not found. skipping...
Read antpos from etcd


## Connect to SNAP2 boards

In [2]:
brdnames=['snap01','snap02','snap03','snap04','snap05','snap06','snap07','snap08','snap09','snap10','snap11']

#connect through Jack's fengine etcd interface for everything that can go through etcd
snapbrds = [snap2_fengine.Snap2FengineEtcd(brdname).fpga for brdname in brdnames]  
fpgfile='/home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c.fpg'


2024-06-17 22:12:29,383 - lwa_f.blocks.block:snap01 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,386 - lwa_f.blocks.block:snap02 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,391 - lwa_f.blocks.block:snap03 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,394 - lwa_f.blocks.block:snap04 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,397 - lwa_f.blocks.block:snap05 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,428 - lwa_f.blocks.block:snap06 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,432 - lwa_f.blocks.block:snap07 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,435 - lwa_f.blocks.block:snap08 - INFO - eth - Couldn't find Ethernet core. Will retry later
2024-06-17 22:12:29,439 - lwa_f.blocks.block:snap09 - INFO - eth - Couldn't find Ethernet core. 

## Completely reset trigger on all boards to make sure there aren't already trigger and veto settings

In [3]:


veto_roles_array=np.zeros(64,dtype=int)
core_roles_array=np.zeros(64,dtype=int)
for p in range(11):
    snapbrd=snapbrds[p]
    setup_coincidencer(snapbrd,0,
                          0,
                          0,
                          0,
                          0,
                          0,
                          core_roles_array,
                          veto_roles_array)
    setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0)


## Set up ethernet and send a test snapshot
Ethernet setup doesn't go through etcd yet.

In [12]:
#connect to the SNAP2s with casperfpga since some functions (ethernet setup) doesn't go through etcd yet
casperbrds = [casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport) for brdname in brdnames]


In [13]:
computer='gpu09' # 'lwacr' or 'gpu09'
packetwait=100 #clock cycles between packets
wait_after_readout=200 #time [clock cycles] to wait for other boards to finish readout
for i,brdname in enumerate(brdnames):
    casperbrd=casperbrds[i]
    #prepare to send a snapshot
    
    setup_data_source(casperbrd,"adc") #set the data source to ADC
    setup_ethernet(brdname,casperbrd,fpgfile,computer,packetwait) #set up the ethernet packetizer and ethernet block
    setvalue(casperbrd,'wait_after_readout','cr_registers.xlsx',wait_after_readout) #set the wait time after readout before new internally-generated triggers are accepted
    setvalue(casperbrd,'brd_id','cr_registers.xlsx',int(brdname[-2:])) #set the board id
    setvalue(casperbrd,'eth_enable','cr_registers.xlsx',1)  #enable ethernet

Data source set to adc
------------------------
snap01:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:01:01:01
Gateway:  0.0.0.1
IP:  10.41.0.201
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0. 25: MAC: 04:3F:72:DF:C1:F0
Setting wait between packets to 100
Data source set to adc
------------------------
snap02:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:02:02:02
Gateway:  0.0.0.1
IP:  10.41.0.202
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips: []
ARP Table: 
IP: 10.41.0. 25: MAC: 04:3F:72:DF:C1:F0
Setting wait between packets to 100
Data source set to adc
------------------------
snap03:cosmic_ray_cr_forty_gbe configuration:
MAC:  02:02:02:03:03:03
Gateway:  0.0.0.1
IP:  10.41.0.203
Fabric port: 
11111
Fabric interface is currently: Enabled
	base_ip: 255.255.255.255
	ip_mask: 255.255.255.255
	rx_ips:

In [15]:
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]

#send trigger to just one board and all will send packets
software_trigger(snapbrds[1],1)

Packetizer formed 64 packets
Board successfully returned to listening state after readout
Readout took 137669 clock cycles
Packetizer never overflowed
Ethernet block almost-full counter incremented by 0


## Run Trigger 

In [52]:
#configure trigger
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]


current_ok_vetos=np.load('/home/ubuntu/kp/ok_vetos-2024-4-26.npy')
current_ok_core=np.load('/home/ubuntu/kp/ok_core-2024-4-26.npy')


ok_snaps=[0,1,2,3,4,5,6,7,8,9,10] #zero indexed
#set up coincidencer
trigger_power_thresh=250**2
veto_power_thresh=175**2
trigger_window=int(round(1.96e8*100/3e8,1)) #number of clock cycles to travel 100 meters
veto_window=int(round(1.96e8*2000/3e8,1)) #number of clock cycles to travel 2000 meters
antenna_number_thresh=10
veto_number_thresh=2

bufferwait=2000

for b in ok_snaps:
    snapbrd=snapbrds[b]
    veto_roles_array=current_ok_vetos[b,:]
    core_roles_array=current_ok_core[b,:]

    setup_coincidencer(snapbrd,trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          core_roles_array,
                          veto_roles_array)
    setvalue(snapbrd,"delay_trigger","cr_registers.xlsx",bufferwait)



In [53]:
#set trigger on or off
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",1) for snapbrd in snapbrds] 

[None, None, None, None, None, None, None, None, None, None, None]

In [54]:
#send a test trigger to check that settings are applied correctly

#send trigger to just one board and all will send packets
#software_trigger(snapbrds[3],1)

## Run Rate Monitor

In [ ]:
%%capture output
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",1) for snapbrd in snapbrds] 

time.sleep(12600) # set a wait time so that running the rate monitor aligns with when I want to save data
now=time.time()
duration=10*60*60 #in seconds
finish=now+duration
waittime=30

core_triggers=np.zeros((11,int(duration/waittime)))
veto_triggers=np.zeros((11,int(duration/waittime)))
core_cycles=np.zeros((11,int(duration/waittime)))
veto_cycles=np.zeros((11,int(duration/waittime)))
successful_triggers=np.zeros((11,int(duration/waittime)))
readout_dones=np.zeros((11,int(duration/waittime)))


i=0
tries=10

time.sleep(waittime)
while time.time()<finish:
    for b in ok_snaps:
        for attempt in range(tries):
            try:
                snapbrd=snapbrds[b]
                veto_coincidence,core_coincidence=snapbrd.read_list_from_ram("cosmic_ray_core_and_veto_rate",2,'u4')
                veto_total_cycles,core_total_cycles=snapbrd.read_list_from_ram("cosmic_ray_core_n_veto_tot_clk_cycles",2,'u4')
                readout_done,successful_trigger=snapbrd.read_list_from_ram("cosmic_ray_core_n_veto_tot_clk_cycles1",2,'u4')
                core_triggers[b,i]=core_coincidence            
                veto_triggers[b,i]=veto_coincidence
                core_cycles[b,i]=core_total_cycles
                veto_cycles[b,i]=veto_total_cycles
                successful_triggers[b,i]=successful_trigger
                readout_dones[b,i]=readout_done
            except:
                if attempt<tries-1:
                    continue
                else:
                    raise
            break
            
    time.sleep(waittime)
    i+=1

#turn trigger off
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]




In [42]:
output.show()

[None, None, None, None, None, None, None, None, None, None, None]

In [56]:
print(i)


1099


In [50]:
[setvalue(snapbrd,"enable_coinc_trig","cr_registers.xlsx",0) for snapbrd in snapbrds]


[None, None, None, None, None, None, None, None, None, None, None]

In [58]:
#save results
fname_prefix='/home/ubuntu/kp/data/2024May15/'
np.save(fname_prefix+'core',np.asarray(core_triggers))
np.save(fname_prefix+'veto',np.asarray(veto_triggers))
np.save(fname_prefix+'vetocycles',np.asarray(veto_cycles))
np.save(fname_prefix+'corecycles',np.asarray(core_cycles))
np.save(fname_prefix+'successfultrigger',np.asarray(successful_triggers))
np.save(fname_prefix+'readoutdone',np.asarray(readout_dones))

In [200]:
print(np.max(core_triggers))
print(core_triggers.shape)
print(8*60*60/30)
print(i)
print(int(duration/waittime))
print(i,(i*waittime)/(60*60))

1919490.0
(11, 960)
960.0
912
960
912 7.6
